In [1]:
#imports
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import regex as re

In [2]:
def gettable(year):
    #3 columns of information
    mtgdates = []
    mtgnames = []
    labels = []
    files = []
    for mo in range (1,13):
        url = 'http://laschoolboard.org/event/select?date_filter%5Bvalue%5D%5Byear%5D={}&date_filter%5Bvalue%5D%5Bmonth%5D={}'.format(year, mo) 
        r = requests.get(url)
        #looking for the container that has the table with all the information 
        soup = BeautifulSoup(r.content)
        a = soup.find('body').find('div', id = 'middlecontainer').find('div', id = 'squeeze-content').find('div').find('table', class_ = 'views-table cols-3').find('tbody')

        #loop through each row in the table
        for x in a.find_all('tr'):
            #mtgdate and name all have consistent class names
            mtgdate = x.find('td', class_ = 'views-field views-field-unix-event-start').get_text().strip()
            mtgname = x.find('td', class_ = 'views-field views-field-title').get_text().strip()
    
            #find links and loop through, add to list of links
            m = x.find('td', class_ = 'views-field views-field-all-files')
            names = []
            links = []
            findlinks = m.find_all('a')
            for link in findlinks:
                if "minutes" in link.text.lower():
                    #get name of pdf
                    names.append(link.text)
                    #get href tag to get directly to the link of the pdf
                    #can use these links later to call the actual data
                    links.append(link.get('href'))
                else:
                    continue
            #add row to the list
            mtgdates.append(mtgdate)
            mtgnames.append(mtgname)
            labels.append(names)
            files.append(links)
    #make dataframe out of all the rows - one dataframe represents one year
    return pd.DataFrame({'mtgdates': mtgdates, 'mtgnames': mtgnames,'names':labels, 'files': files})

In [12]:
#code from https://stackoverflow.com/questions/22800100/parsing-a-pdf-via-url-with-python-using-pdfminer
import urllib
import urllib.error
import regex as re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from io import BytesIO

def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    f = urllib.request.urlopen(url).read()
    fp = BytesIO(f)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 50
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

def application (pdfurls):
    texts = []
    if len(pdfurls) > 0:
        for pdfurl in pdfurls:
            print(pdfurl)
            text = pdf_from_url_to_txt(re.sub(r" ", "%20", pdfurl))
            text_wordsonly = re.sub(r"[^A-z\s]", "", text)
            text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
            texts.append(text_wordsonly)
            #adds all the text to file - problem- if rerun does not overwrite file - need to fix this
            with open("{}.txt".format(year), "a+") as text_file:
                text_file.write(text_wordsonly + "\n")
    return texts
#http://laschoolboard.org/sites/default/files/127%20(Charter)%20Approval%20of%20Animo%20Charter%20MS1.pdf

In [76]:
mainframe = pd.DataFrame()
for year in range(2010,2022):
    mainframe = pd.concat([mainframe, gettable(year)])
mainframe

,mtgdates,mtgnames,names,files
0,"Tuesday, January 5, 2010 - 11:00am",Special Board Closed Session 1-5-10,[01-05-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...
1,"Tuesday, January 5, 2010 - 3:30pm",Special Board Meeting 1-5-10,[01-5-10 Special Board Minutes.pdf],[http://laschoolboard.org/sites/default/files/...
2,"Tuesday, January 12, 2010 - 11:00am",Special Board Closed Session 1-12-10,[01-12-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...
3,"Tuesday, January 12, 2010 - 2:00pm",Regular Board Meeting 1/12/10,[01-12-10 Minutes],[http://laschoolboard.org/sites/default/files/...
4,"Monday, January 18, 2010 - 2:00pm",Committee of the Whole 1-18-11,[],[]
...,...,...,...,...
96,"Thursday, December 9, 2021 - 10:00am",CANCELLED: Committee of the Whole - December 9...,[],[]
97,"Thursday, December 9, 2021 - 6:00pm","Family and Community Engagement - Thursday, De...",[],[]
98,"Tuesday, December 14, 2021 - 10:00am","Annual Board Meeting - December 14, 2021 - 9:0...",[],[]
99,"Tuesday, December 14, 2021 - 10:30am","Regular Board Meeting - December 14, 2021 - 9:...",[],[]


In [82]:
def gettext(pdfurl):
    try:
        text = pdf_from_url_to_txt(re.sub(r" ", "%20", pdfurl))
        text_wordsonly = re.sub(r"[^A-z\s]", "", text)
        text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
        text_wordsonly = text_wordsonly.lower()
        return text_wordsonly
    except urllib.error.HTTPError as err:
        return "error in {}".format(pdfurl)

In [83]:
wordlist = []
for pdflist in mainframe.files:
    wordlist.append([gettext(re.sub(r" ", "%20", pdfurl)) for pdfurl in pdflist])
    #wordlist.append(([re.sub(r"\s+", " ",re.sub(r"[^A-z\s]", "",pdf_from_url_to_txt(re.sub(r" ", "%20", pdfurl)))) for pdfurl in pdflist]))
mainframe['words'] = wordlist
mainframe

,mtgdates,mtgnames,names,files,words
0,"Tuesday, January 5, 2010 - 11:00am",Special Board Closed Session 1-5-10,[01-05-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
1,"Tuesday, January 5, 2010 - 3:30pm",Special Board Meeting 1-5-10,[01-5-10 Special Board Minutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
2,"Tuesday, January 12, 2010 - 11:00am",Special Board Closed Session 1-12-10,[01-12-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
3,"Tuesday, January 12, 2010 - 2:00pm",Regular Board Meeting 1/12/10,[01-12-10 Minutes],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
4,"Monday, January 18, 2010 - 2:00pm",Committee of the Whole 1-18-11,[],[],[]
...,...,...,...,...,...
96,"Thursday, December 9, 2021 - 10:00am",CANCELLED: Committee of the Whole - December 9...,[],[],[]
97,"Thursday, December 9, 2021 - 6:00pm","Family and Community Engagement - Thursday, De...",[],[],[]
98,"Tuesday, December 14, 2021 - 10:00am","Annual Board Meeting - December 14, 2021 - 9:0...",[],[],[]
99,"Tuesday, December 14, 2021 - 10:30am","Regular Board Meeting - December 14, 2021 - 9:...",[],[],[]


In [84]:
mainframe.to_pickle('alltext.pandas')

In [85]:
df = pd.read_pickle('alltext.pandas')
df

,mtgdates,mtgnames,names,files,words
0,"Tuesday, January 5, 2010 - 11:00am",Special Board Closed Session 1-5-10,[01-05-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
1,"Tuesday, January 5, 2010 - 3:30pm",Special Board Meeting 1-5-10,[01-5-10 Special Board Minutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
2,"Tuesday, January 12, 2010 - 11:00am",Special Board Closed Session 1-12-10,[01-12-10CSMinutes.pdf],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
3,"Tuesday, January 12, 2010 - 2:00pm",Regular Board Meeting 1/12/10,[01-12-10 Minutes],[http://laschoolboard.org/sites/default/files/...,[board of education of the city of los angeles...
4,"Monday, January 18, 2010 - 2:00pm",Committee of the Whole 1-18-11,[],[],[]
...,...,...,...,...,...
96,"Thursday, December 9, 2021 - 10:00am",CANCELLED: Committee of the Whole - December 9...,[],[],[]
97,"Thursday, December 9, 2021 - 6:00pm","Family and Community Engagement - Thursday, De...",[],[],[]
98,"Tuesday, December 14, 2021 - 10:00am","Annual Board Meeting - December 14, 2021 - 9:0...",[],[],[]
99,"Tuesday, December 14, 2021 - 10:30am","Regular Board Meeting - December 14, 2021 - 9:...",[],[],[]


### UPDATES

Still some errors but getting better
> BASICALLY before was only reading from january but i caught and fixed that mistake 

> now its a HUGE database - so i limited to just board minutes - still huge file

> these were all issues before limiting to minutes -might still need to be adressed
>> btw some files are unreadable - but I dont think thats fixable, they seem scanned in 

>> need to catch and handle exception that happens in 2011 and 2016 

>>some of the files are really long and take a long time (1000 + pages) - limit to 50 pages


### Next Steps
- sam - everything else 
- REMOVE STOP WORDS 
- general analysis - ngrams,  bag of words
- LDA
- search for transporation issues
    - ngrams
    - list of transportation words (bus, school bus, MORE WORDS)
    - collocations in future
- analyze text by year

In [62]:
for text in texts:
    print(len(text))

0


In [14]:
#df.to_pickle('links12.pandas')